In [1]:
import os
print(os.listdir("../input"))
import pandas as pd
import numpy as np
import cv2 
from glob import glob
from sklearn import preprocessing
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
#packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
#allows charts to appear in the notebook 
%matplotlib inline
#import OS to share the project dir
#os.chdir('H:\Barq')
#ignoring warnings
import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

['train', 'test', 'sample_submission.csv']


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BASE_DATA_FOLDER = "../input"
TRAIN_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")
sizeAfterResize=(32,32)
x_train = [] 
y_train = []


#### data diemensions

In [3]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 32

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 12

In [4]:
def img_preproc(img):
        ## convert to hsv 
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            
        ## mask of green (36,0,0) ~ (70, 255,255) #(25, 0, 10), (70, 255,255)
        #best (25, 40, 30), (95, 255,255)
        mask = cv2.inRange(hsv, (25, 40, 50), (75, 255, 255))
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        ## slice the green
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        green=sharpen_image(green)

        #converting to gray scale to apply contour
        gray = cv2.cvtColor(green,cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
        contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        #doing rectangle on x ,y,w,h
        x,y,w,h = cv2.boundingRect(cnt)
        #getting the cropped image 
        crop = green[y:y+h,x:x+w]
        return crop

# 4-Segmentation

In [5]:
#apply gaussian 
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [6]:
images_per_class = {}
labels=['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat'
        ,'Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed'
        ,'Shepherds Purse','Small-flowered Cranesbill','Sugar beet']
# get all classes name
for class_folder_name in os.listdir(TRAIN_DATA_FOLDER):
    class_folder_path = os.path.join(TRAIN_DATA_FOLDER, class_folder_name)
    class_label = class_folder_name
    images_per_class[class_label] = []
    print("reading data from class:",class_label)
    #get the images in each class
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        images_per_class[class_label].append(image)

reading data from class: Maize
reading data from class: Fat Hen
reading data from class: Common Chickweed
reading data from class: Cleavers
reading data from class: Charlock
reading data from class: Loose Silky-bent
reading data from class: Small-flowered Cranesbill
reading data from class: Black-grass
reading data from class: Scentless Mayweed
reading data from class: Sugar beet
reading data from class: Common wheat
reading data from class: Shepherds Purse


In [7]:

print  ("Read My Data Read My" ) 
for j in range(12):
    count=0
    for img in images_per_class[labels[j]] :
            count=count+1
            crop =img_preproc(img)
            #get the resized image
            if len(crop)!=0 :
                resized= cv2.resize(crop, sizeAfterResize)
                ## save my new image
                image_name = labels[j]+"."+str(count)+".png"
                print("write the image number "+str(count))
                #cv2.imwrite(os.path.join(RESULT_DATA_FOLDER ,image_name),resized)
                #append my data
                x_train.append(np.array(resized))
                y_train.append(j)    
            else:
                x_train.append(np.array(crop))
                y_train.append(j)   
            
            
print("Done")

Read My Data Read My
write the image number 1
write the image number 2
write the image number 3
write the image number 4
write the image number 5
write the image number 6
write the image number 7
write the image number 8
write the image number 9
write the image number 10
write the image number 11
write the image number 12
write the image number 13
write the image number 14
write the image number 15
write the image number 16
write the image number 17
write the image number 18
write the image number 19
write the image number 20
write the image number 21
write the image number 22
write the image number 23
write the image number 24
write the image number 25
write the image number 26
write the image number 27
write the image number 28
write the image number 29
write the image number 30
write the image number 31
write the image number 32
write the image number 33
write the image number 34
write the image number 35
write the image number 36
write the image number 37
write the image number 38


write the image number 51
write the image number 52
write the image number 53
write the image number 54
write the image number 55
write the image number 56
write the image number 57
write the image number 58
write the image number 59
write the image number 60
write the image number 61
write the image number 62
write the image number 63
write the image number 64
write the image number 65
write the image number 66
write the image number 67
write the image number 68
write the image number 69
write the image number 70
write the image number 71
write the image number 72
write the image number 73
write the image number 74
write the image number 75
write the image number 76
write the image number 77
write the image number 78
write the image number 79
write the image number 80
write the image number 81
write the image number 82
write the image number 83
write the image number 84
write the image number 85
write the image number 86
write the image number 87
write the image number 88
write the im

write the image number 361
write the image number 362
write the image number 363
write the image number 364
write the image number 365
write the image number 366
write the image number 367
write the image number 368
write the image number 369
write the image number 370
write the image number 371
write the image number 372
write the image number 373
write the image number 374
write the image number 375
write the image number 376
write the image number 377
write the image number 378
write the image number 379
write the image number 380
write the image number 381
write the image number 382
write the image number 383
write the image number 384
write the image number 385
write the image number 386
write the image number 387
write the image number 388
write the image number 389
write the image number 390
write the image number 1
write the image number 2
write the image number 3
write the image number 4
write the image number 5
write the image number 6
write the image number 7
write the image

write the image number 20
write the image number 21
write the image number 22
write the image number 23
write the image number 24
write the image number 25
write the image number 26
write the image number 27
write the image number 28
write the image number 29
write the image number 30
write the image number 31
write the image number 32
write the image number 33
write the image number 34
write the image number 35
write the image number 36
write the image number 37
write the image number 38
write the image number 39
write the image number 40
write the image number 41
write the image number 42
write the image number 43
write the image number 44
write the image number 45
write the image number 46
write the image number 47
write the image number 48
write the image number 49
write the image number 50
write the image number 51
write the image number 52
write the image number 53
write the image number 54
write the image number 55
write the image number 56
write the image number 57
write the im

write the image number 354
write the image number 355
write the image number 356
write the image number 357
write the image number 358
write the image number 359
write the image number 360
write the image number 361
write the image number 362
write the image number 363
write the image number 364
write the image number 365
write the image number 366
write the image number 367
write the image number 368
write the image number 369
write the image number 370
write the image number 371
write the image number 372
write the image number 373
write the image number 374
write the image number 375
write the image number 376
write the image number 377
write the image number 378
write the image number 379
write the image number 380
write the image number 381
write the image number 382
write the image number 383
write the image number 384
write the image number 385
write the image number 386
write the image number 387
write the image number 388
write the image number 389
write the image number 390
w

...

write the image number 89
write the image number 90
write the image number 91
write the image number 92
write the image number 93
write the image number 94
write the image number 95
write the image number 96
write the image number 97
write the image number 98
write the image number 99
write the image number 100
write the image number 101
write the image number 102
write the image number 103
write the image number 104
write the image number 105
write the image number 106
write the image number 107
write the image number 108
write the image number 109
write the image number 110
write the image number 111
write the image number 112
write the image number 113
write the image number 114
write the image number 115
write the image number 116
write the image number 117
write the image number 118
write the image number 119
write the image number 120
write the image number 121
write the image number 122
write the image number 123
write the image number 124
write the image number 125
write the im

write the image number 406
write the image number 407
write the image number 408
write the image number 409
write the image number 410
write the image number 411
write the image number 412
write the image number 413
write the image number 414
write the image number 415
write the image number 416
write the image number 417
write the image number 418
write the image number 419
write the image number 420
write the image number 421
write the image number 422
write the image number 423
write the image number 424
write the image number 425
write the image number 426
write the image number 427
write the image number 428
write the image number 429
write the image number 430
write the image number 431
write the image number 432
write the image number 433
write the image number 434
write the image number 435
write the image number 436
write the image number 437
write the image number 438
write the image number 439
write the image number 440
write the image number 441
write the image number 442
w

write the image number 210
write the image number 211
write the image number 212
write the image number 213
write the image number 214
write the image number 215
write the image number 216
write the image number 217
write the image number 218
write the image number 219
write the image number 220
write the image number 221
write the image number 222
write the image number 223
write the image number 224
write the image number 225
write the image number 226
write the image number 227
write the image number 228
write the image number 229
write the image number 230
write the image number 231
write the image number 1
write the image number 2
write the image number 3
write the image number 4
write the image number 5
write the image number 6
write the image number 7
write the image number 8
write the image number 9
write the image number 10
write the image number 11
write the image number 12
write the image number 13
write the image number 14
write the image number 15
write the image number 16

write the image number 305
write the image number 306
write the image number 307
write the image number 308
write the image number 309
write the image number 310
write the image number 311
write the image number 312
write the image number 313
write the image number 314
write the image number 315
write the image number 316
write the image number 317
write the image number 318
write the image number 319
write the image number 320
write the image number 321
write the image number 322
write the image number 323
write the image number 324
write the image number 325
write the image number 326
write the image number 327
write the image number 328
write the image number 329
write the image number 330
write the image number 331
write the image number 332
write the image number 333
write the image number 334
write the image number 335
write the image number 336
write the image number 337
write the image number 338
write the image number 339
write the image number 340
write the image number 341
w

write the image number 119
write the image number 120
write the image number 121
write the image number 122
write the image number 123
write the image number 124
write the image number 125
write the image number 126
write the image number 127
write the image number 128
write the image number 129
write the image number 130
write the image number 131
write the image number 132
write the image number 133
write the image number 134
write the image number 135
write the image number 136
write the image number 137
write the image number 138
write the image number 139
write the image number 140
write the image number 141
write the image number 142
write the image number 143
write the image number 144
write the image number 145
write the image number 146
write the image number 147
write the image number 148
write the image number 149
write the image number 150
write the image number 151
write the image number 152
write the image number 153
write the image number 154
write the image number 155
w

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, 
                                                test_size=0.15, random_state=7, 
                                                stratify = y_train)

In [9]:
# x_train = []
# y_train = [] 
# print ("Read My Segmented Data") 
# for img in glob(os.path.join(RESULT_DATA_FOLDER, "*.png")):
#             x_train.append(np.array(img))
#             label = img.split(os.path.sep)[-1].split(".")[0]
#             y_train.append(label)
# print("Done")
print(y_train)

[3, 10, 3, 10, 8, 2, 3, 6, 8, 3, 10, 3, 6, 1, 3, 6, 8, 3, 6, 1, 11, 11, 1, 8, 4, 5, 3, 10, 5, 10, 10, 7, 5, 9, 8, 2, 4, 6, 0, 5, 9, 9, 10, 7, 3, 5, 6, 8, 3, 2, 1, 10, 3, 2, 10, 0, 0, 1, 6, 10, 9, 3, 3, 8, 8, 8, 6, 7, 10, 11, 2, 5, 8, 3, 6, 6, 11, 8, 6, 8, 3, 3, 8, 10, 11, 6, 10, 10, 11, 3, 9, 8, 8, 5, 5, 1, 6, 8, 10, 6, 1, 11, 6, 7, 10, 7, 5, 5, 3, 0, 3, 3, 11, 5, 3, 0, 6, 6, 8, 10, 10, 7, 3, 5, 6, 11, 8, 10, 6, 3, 0, 11, 7, 10, 3, 6, 5, 11, 5, 9, 3, 4, 6, 3, 8, 11, 11, 7, 10, 3, 0, 7, 6, 9, 0, 6, 7, 5, 5, 1, 10, 8, 0, 2, 8, 0, 1, 10, 8, 9, 2, 6, 7, 5, 8, 11, 8, 8, 3, 1, 6, 4, 4, 6, 1, 8, 6, 11, 11, 2, 9, 2, 5, 3, 6, 8, 11, 8, 0, 11, 0, 7, 0, 10, 1, 0, 10, 3, 8, 11, 10, 6, 0, 4, 7, 8, 9, 1, 1, 8, 11, 1, 6, 2, 6, 2, 8, 8, 5, 4, 3, 9, 10, 10, 2, 5, 10, 2, 3, 10, 6, 6, 9, 0, 8, 5, 2, 8, 11, 5, 3, 11, 3, 0, 3, 6, 6, 5, 3, 8, 1, 3, 6, 9, 6, 6, 3, 3, 1, 1, 8, 8, 6, 11, 4, 5, 1, 11, 11, 6, 2, 6, 0, 1, 10, 11, 8, 4, 2, 2, 2, 1, 6, 4, 3, 7, 7, 11, 1, 5, 8, 5, 6, 7, 5, 1, 10, 9, 6, 6, 11, 11, 4,

In [10]:
len(x_train)

4037

### Helper Function to create new weights and biases

In [11]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

### Helper Function to create new CONV Layer

In [12]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

### helper function to flat images

In [13]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

### helper function to create fully connected layer

In [14]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

#### define place holders for our model

In [15]:
x = tf.placeholder(tf.float32, shape=[None,3, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

In [16]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [17]:
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


## Creating our CNN 

In [18]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=5,
                   num_filters=16,
                   use_pooling=True)

In [19]:
layer_flat, num_features = flatten_layer(layer_conv1)

In [20]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=128,
                         use_relu=True)

## predicted class

In [21]:
y_pred = tf.nn.softmax(layer_fc1)
y_pred_cls = tf.argmax(y_pred, dimension=1)
print(y_pred_cls)

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)


## Cost Function

In [22]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc1,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Optimization Method

In [23]:
optimizer = tf.train.AdamOptimizer(learning_rate=.0001).minimize(cost)

####  measure accuracy

In [24]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### start session

In [25]:
session = tf.Session()

In [26]:
session.run(tf.global_variables_initializer())

### optimizer

In [27]:
train_batch_size = 64

In [28]:
# Counter for total number of iterations performed so far.
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = x_train.next_batch(train_batch_size)

        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

        # Print status every 100 iterations.
        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))

    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

## plot confusion matrix

In [29]:
def plot_confusion_matrix(cls_pred):
    # This is called from print_test_accuracy() below.

    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Get the true classifications for the test-set.
    cls_true = y_test
    
    # Get the confusion matrix using sklearn.
   # cm = confusion_matrix(y_true=cls_true, y_pred=cls_pred)

    # Print the confusion matrix as text.
    #print(cm)

    # Plot the confusion matrix as an image.
    #plt.matshow(cm)

    # Make various adjustments to the plot.
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

# Test Accuracy

In [30]:
# Split the test-set into smaller batches of this size.
test_batch_size = 256
images_size=0
def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # Number of images in the test-set.
    num_test = len(x_test)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0
    
    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = x_test[i:j]
        images_size=len(images)
        #images= np.reshape(images, ( 1024)).T
        # Get the associated labels.
        labels = y_train[i:j]
        Hot_Encoded_Labels = np.zeros ([len(labels),12])
        for i in range (len(labels)):
            Hot_Encoded_Labels[i,int(labels[i]-1)] = 1
        print(Hot_Encoded_Labels)
        imgs=[]
        for k in images:
           k=k.reshape(-1,1024)
           imgs.append(k)
        print(k.shape)
        # Create a feed-dict with these images and labels.
        feed_dict = {x: imgs,
                     y_true: Hot_Encoded_Labels}
        # Calculate the predicted class using TensorFlow.
        print (y_true.shape)
#         cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        pred= session.run(y_pred_cls, feed_dict=feed_dict)


        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = y_test

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [31]:
print_test_accuracy()

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3, 1024)
(?, 12)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
(3, 1024)
(?, 12)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3, 1024)
(?, 12)
Accuracy on Test-Set: 5.6% (40 / 713)


  # Spliting data for training and test

# Data Augmentation